<h1 align=center><font size = 7>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Install and import necessary libraries 

In [1]:
! pip install beautifulsoup4 as bs4
! pip install geocoder
!conda install -c conda-forge geopy --yes # uncomment this line if installation is required

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from pandas import json_normalize # tranform JSON file into a pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup

import geocoder # import geocoder

import folium # map rendering library

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json # library to handle JSON files
import requests # library to handle requests

# import k-means 
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


     |████████████████████████████████| 122kB 19.6MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 102kB 5.6MB/s ta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

  geographiclib      cond

## Store url for downloading wikipedia webpage with Toronto postal codes and convert postal code table into dataframe

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# read html from webpage via pandas
df=pd.read_html(url, flavor='html5lib')

In [4]:
# convert list to dataframe
df=pd.DataFrame(df[0])
df.shape

(180, 3)

In [5]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Drop rows with postal codes which are not assigned to any Borough

In [6]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True)
df.shape

(103, 3)

In [7]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Find rows with postal code which do not have any neighbourhoods assigned

In [8]:
df[df['Neighbourhood']== 'Not Assigned'].index.tolist()

[]

No unassigned neighbourhoods found

## Find postal codes which are repeated and combine their neighbourhoods

In [9]:
df['Postal Code'].value_counts().sort_values(ascending = False)

M9A    1
M7A    1
M9W    1
M4H    1
M9L    1
M4Y    1
M5S    1
M3J    1
M2H    1
M8Y    1
M6L    1
M3H    1
M4K    1
M5A    1
M2R    1
M6S    1
M2K    1
M3N    1
M1M    1
M4W    1
M9R    1
M2N    1
M9C    1
M4E    1
M1S    1
M3C    1
M3B    1
M4P    1
M4M    1
M1L    1
M1W    1
M1K    1
M8X    1
M4C    1
M6R    1
M4A    1
M5M    1
M5B    1
M5R    1
M6M    1
M1J    1
M6A    1
M2M    1
M5P    1
M5L    1
M8V    1
M5E    1
M1X    1
M4R    1
M5C    1
M4X    1
M5V    1
M3L    1
M5J    1
M5T    1
M9V    1
M1B    1
M9N    1
M2J    1
M4S    1
M1T    1
M7R    1
M4N    1
M3K    1
M4L    1
M6K    1
M1R    1
M7Y    1
M5X    1
M5K    1
M4B    1
M6C    1
M9M    1
M2L    1
M6H    1
M6N    1
M4J    1
M6P    1
M1P    1
M6J    1
M8Z    1
M8W    1
M3A    1
M4V    1
M1V    1
M1G    1
M9P    1
M2P    1
M1N    1
M9B    1
M5N    1
M4T    1
M1C    1
M5W    1
M6G    1
M3M    1
M6B    1
M6E    1
M1E    1
M1H    1
M5G    1
M5H    1
M4G    1
Name: Postal Code, dtype: int64

Each postal code is found only once so no need to combine neighbourhoods

In [10]:
# reset index of data frame, throw away old indices
df.reset_index(drop=True)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [11]:
df.shape

(103, 3)

<h1 align=center><font size = 5>Add location data of neighbourhoods</font></h1>

## Download and read csv with location information of neighbourhoods

In [12]:
# initialize your variable to None
lat_lng_coords = None

In [13]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [14]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
# check size of new dataframe
geo_df.shape

(103, 3)

In [16]:
# check if any postal codes are repeated
df['Postal Code'].value_counts()

M4G    1
M4M    1
M1L    1
M1W    1
M1K    1
M8X    1
M4C    1
M6R    1
M4A    1
M5M    1
M5B    1
M4P    1
M5R    1
M1J    1
M6A    1
M2M    1
M5P    1
M5L    1
M8V    1
M5E    1
M1X    1
M4R    1
M5C    1
M6M    1
M3C    1
M5A    1
M1S    1
M9W    1
M4H    1
M9L    1
M4Y    1
M5S    1
M3J    1
M2H    1
M8Y    1
M6L    1
M3H    1
M7A    1
M4K    1
M2R    1
M6S    1
M2K    1
M3N    1
M1M    1
M4W    1
M9R    1
M2N    1
M9C    1
M4E    1
M4X    1
M3B    1
M5V    1
M1P    1
M8Z    1
M8W    1
M3A    1
M4V    1
M1V    1
M1G    1
M9P    1
M2P    1
M1N    1
M9B    1
M6J    1
M5N    1
M1C    1
M5W    1
M6G    1
M3M    1
M6B    1
M6E    1
M1E    1
M1H    1
M5G    1
M5H    1
M4T    1
M6P    1
M6K    1
M4J    1
M5T    1
M9V    1
M1B    1
M9N    1
M2J    1
M4S    1
M1T    1
M7R    1
M4N    1
M3K    1
M5J    1
M4L    1
M1R    1
M7Y    1
M5X    1
M5K    1
M4B    1
M6C    1
M9M    1
M2L    1
M6H    1
M6N    1
M3L    1
M9A    1
Name: Postal Code, dtype: int64

## Combine the two dataframes, postal code column is present in both, use it as the key to merge

In [17]:
combined_df = pd.merge(df, geo_df, on='Postal Code')

In [18]:
# check size of combined dataframe
combined_df.shape

(103, 5)

In [19]:
# Print out data frame to confirm that merge was done correctly
combined_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h1 align=center><font size = 5>Explore and cluster the neighborhoods in Toronto</font></h1>

## Map downtown toronto and place markers for all postal codes

In [20]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for postcode, lat, lng, borough, neighbourhood in zip(combined_df['Postal Code'], combined_df['Latitude'], combined_df['Longitude'], combined_df['Borough'], combined_df['Neighbourhood']):
    label = '{}, {}, {}'.format(postcode, borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [22]:
# setup variables to access Foursquare data
CLIENT_ID = '23SL1DGVQLLNGH3OQXHBCEU2GGC2STKCJ05WQCE2YMQJA5PT' # your Foursquare ID
CLIENT_SECRET = 'YXHZ5OKVF0X2YTFWVBZNQW03WYSOCFTFWBQLJ4E0UKK3VLBW' # your Foursquare Secret
VERSION = '20210207' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = 'MAP0CGTV3V4DKK212Y4OMYHUH0ZQZZ4ME22YCLNCY55QJTAZ' # your FourSquare Access Token

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 23SL1DGVQLLNGH3OQXHBCEU2GGC2STKCJ05WQCE2YMQJA5PT
CLIENT_SECRET:YXHZ5OKVF0X2YTFWVBZNQW03WYSOCFTFWBQLJ4E0UKK3VLBW


Create a function to find venues closeby in Toronto postal codes using FourSquare API

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000): # Radius increased to find meaningful number of venues
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Step through all postal codes location and save returned venues

In [24]:
toronto_venues = getNearbyVenues(names=combined_df['Postal Code'],
                                   latitudes=combined_df['Latitude'],
                                   longitudes=combined_df['Longitude']
                                  )

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(4889, 7)


,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,M3A,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,M3A,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
4,M3A,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant


Group venues by postal code to find out number of venues found near each postal code location

In [26]:
toronto_venues.groupby('Postal Code').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M1B,17,17,17,17,17,17
M1C,5,5,5,5,5,5
M1E,22,22,22,22,22,22
M1G,9,9,9,9,9,9
M1H,27,27,27,27,27,27
M1J,13,13,13,13,13,13
M1K,29,29,29,29,29,29
M1L,25,25,25,25,25,25
M1M,10,10,10,10,10,10


In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 332 uniques categories.


Encode venue data to prepare for clustering 

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smooth

In [29]:
toronto_onehot.shape

(4889, 333)

In [30]:
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped

,Postal Code,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smooth

In [31]:
toronto_grouped.shape

(102, 333)

Run kmeans clustering algorithm

In [32]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1) #clustering requires numeric value only so remove postal code

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 4, 0, 4, 4, 4, 4, 4, 0], dtype=int32)

Merge cluster labels with postal codes and location data

In [33]:
toronto_merged = pd.DataFrame(toronto_grouped['Postal Code'])

In [34]:
# add clustering labels
toronto_merged.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged.head()

,Cluster Labels,Postal Code
0,4,M1B
1,0,M1C
2,4,M1E
3,0,M1G
4,4,M1H


In [35]:
# merge to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(combined_df.set_index('Postal Code'), on='Postal Code')

toronto_merged.head() # check the last columns!

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,4,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,0,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,0,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Draw map and place cluster pins with idenfication information

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, bur in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels'], toronto_merged['Borough']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + ' ' + str(bur), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters